In [169]:
import os
import time
import math
import numpy as np
import pandas as pd

In [38]:
# change working directory
os.chdir(os.getcwd())

In [158]:
def date_adjustment(df):
    
    df['Date'] = pd.to_datetime(df['Date'])
    cols = ['Year', 'Month', 'Day']
    cols.extend(df.columns.tolist())
    df['Day'] = df['Date'].dt.day
    df['Month'] = df['Date'].dt.month
    df['Year'] = df['Date'].dt.year
    df.sort_values(by=['Date'], inplace=True)
    df = df.reindex(columns=cols)
    df.drop(['Date'], axis=1, inplace=True)
    
    return df

def new_df(df, instrument_name='SPY'):
    
    df_adj = pd.DataFrame()
    df_adj['Date'] = pd.to_datetime(df['Date'].values)
    df_adj[instrument_name] = df['Adj Close'].values
    #df_adj['SMA30'] = df_adj[instrument_name].rolling(window=30).mean()
    #df_adj['SMA100'] = df_adj[instrument_name].rolling(window=100).mean()
    
    return df_adj

def buy_sell_kadane(signal):
    
    if len(signal) < 2:
        return None
    
    start = time.time()
    
    curr_buy = signal.iloc[0,1]
    glob_sell = signal.iloc[1,1]
    glob_profit = glob_sell - curr_buy
    curr_profit = float('-inf')
    
    curr_buy_time = signal.iloc[0,0]
    glob_sell_time = signal.iloc[1,0]
    res = []
    
    for i in range(1, len(signal)):
        curr_profit = signal.iloc[i,1] - curr_buy
        
        if curr_profit > glob_profit:
            glob_profit = curr_profit
            glob_sell = signal.iloc[i,1]
            glob_sell_time = signal.iloc[i,0]
        
        if curr_buy > signal.iloc[i,1]:
            curr_buy = signal.iloc[i,1]
            curr_buy_time = signal.iloc[i,0]
    
    res.append([curr_buy_time, curr_buy, glob_sell_time, glob_sell, glob_sell - curr_buy])
    res = pd.DataFrame(res, columns = ['Buy Date', 'Buy Amount', 'Sell Date',
                                       'Sell Amount', 'Profit'])
    
    end = time.time()
    time_taken = round(end - start, 3)
    
    return res, time_taken

def buy_sell_brute(signal):
    
    if len(signal) < 2:
        return None
    
    start = time.time()
    res = []
    
    for i in range(0, len(signal)):
        curr_buy = signal.iloc[i,1]
        curr_buy_time = signal.iloc[i,0]
        curr_max_profit = float('-inf')
        
        for j in range(i, len(signal)):
            curr_sell = signal.iloc[j,1]
            curr_profit = curr_sell - curr_buy
            
            if curr_profit > curr_max_profit:
                curr_max_profit = curr_profit
                curr_max_sell = signal.iloc[j,1]
                curr_sell_time = signal.iloc[j,0]
            
            if curr_buy > signal.iloc[j,1]:
                curr_buy = signal.iloc[j,1]
                curr_buy_time = signal.iloc[j,0]
        
        res.append([curr_buy_time, curr_buy, curr_sell_time, curr_max_sell, curr_max_sell - curr_buy])
    
    res = pd.DataFrame(res, columns = ['Buy Date', 'Buy Amount', 'Sell Date',
                                       'Sell Amount', 'Profit'])
    final_res = res[res['Profit']==res['Profit'].max()].drop_duplicates()
    
    end = time.time()
    time_taken = round(end - start, 3)
    
    return final_res, time_taken

def compare_time(signal):
    
    kadane_time = run_kadane(signal, compare=True)
    
    print('')
    time.sleep(2)
    
    brute_time = run_brute(signal, compare=True)
    
    print('')
    
    if kadane_time < brute_time:
        print('Kadane algorithm runs faster.')
        print('Time difference:', round(brute_time-kadane_time,3))
    elif brute_time > kadane_time:
        print('Brute force algorthim runs faster.')
        print('Time difference:', round(kadane_time-brute_time,3))
    else:
        print('Both takes the same amount of time.')
    
    return

def run_kadane(signal, compare=False):
    
    print('Running optimized algorithm . . .')
    kadane_res, kadane_time = buy_sell_kadane(signal)
    print('Buy Date:', kadane_res.iloc[0,0], 'Buy Price:', kadane_res.iloc[0,1])
    print('Sell Date:', kadane_res.iloc[0,2], 'Sell Price:', kadane_res.iloc[0,3])
    print('Total profit:', kadane_res.iloc[0,-1])
    print('Time taken:', round(kadane_time,3))
    
    if compare is True:
        return kadane_time
    else:
        return

def run_brute(signal, compare=False):
    
    print('Running brute force algorithm . . .')
    brute_res, brute_time = buy_sell_brute(signal)
    print('Buy Date:', brute_res.iloc[0,0], 'Buy Price:', brute_res.iloc[0,1])
    print('Sell Date:', brute_res.iloc[0,2], 'Sell Price:', brute_res.iloc[0,3])
    print('Total profit:', brute_res.iloc[0,-1])
    print('Time taken:', round(brute_time,3))
    
    if compare is True:
        return brute_time
    else:
        return

In [ ]:
def pick_instrument():
    
    if choice == 'GLD':
        signal = pd.DataFrame('./data/GLD (Gold Shares 01-01-2008 to 03-07-2022).csv')
    elif choice == 'SGX':
        signal = pd.DataFrame('./data/SGD=X (01-01-2008 to 03-07-2022).csv')
    elif choice == 'SPY':
        signal = pd.DataFrame('./data/SPY (S&P500 ETF 01-01-2008 to 03-07-2022)).csv')
    elif choice == 'TLT':
        signal = pd.DataFrame('./data/TLT (US Treasury Bonds ETF 01-01-2008 to 03-07-2022).csv')
    elif choice == 'USO':
        signal = pd.DataFrame('powerx_capstone1/data/USO (US Oil Index 01-01-2008 to 03-07-2022).csv')